In [19]:
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, SentimentOptions
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from google.cloud import language_v1
import boto3
from dotenv import load_dotenv
import os

load_dotenv()
IBM_API_KEY = os.getenv("IBM_API_KEY")
IBM_URL = os.getenv("IBM_URL")

GOOGLE_API_CREDENTIALS_PATH = os.getenv("GOOGLE_API_CREDENTIALS_PATH")

AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")
AWS_REGION = os.getenv("AWS_REGION")

In [20]:
# IBM Watson NLU Sentiment Analysis
def analyze_sentiment_ibm(text, language):
    authenticator = IAMAuthenticator(IBM_API_KEY)
    nlu = NaturalLanguageUnderstandingV1(
        version="2021-08-01",
        authenticator=authenticator
    )
    nlu.set_service_url(IBM_URL)

    response = nlu.analyze(
        text=text,
        features=Features(sentiment=SentimentOptions()),
        language=language  # Specify the language
    ).get_result()

    sentiment = response['sentiment']['document']['label']
    return sentiment

# Google Cloud NLP Sentiment Analysis
def analyze_sentiment_google(text, language_code):
    client = language_v1.LanguageServiceClient.from_service_account_json(GOOGLE_API_CREDENTIALS_PATH)

    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT, language=language_code)
    response = client.analyze_sentiment(document=document)

    sentiment_score = response.document_sentiment.score
    if sentiment_score > 0:
        return "positive"
    elif sentiment_score < 0:
        return "negative"
    else:
        return "neutral"

# AWS Comprehend Sentiment Analysis
def analyze_sentiment_aws(text, language_code):
    client = boto3.client(
        "comprehend",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION
    )

    response = client.detect_sentiment(
        Text=text,
        LanguageCode=language_code
    )

    sentiment = response['Sentiment']
    return sentiment.lower()  # AWS returns "POSITIVE", "NEGATIVE", etc.

In [21]:
languages = {"en": "English", "es": "Spanish", "zh": "Chinese"}
# Map of languages to their respective sentences
sentences = {
    "en": [
        "The weather is beautiful today.",
        "I love eating pizza on weekends.",
        "She is very talented in playing the piano.",
        "This is the best movie I have ever seen.",
        "The product exceeded my expectations.",
        "I am very disappointed with the service.",
        "The new iPhone is too expensive.",
        "He is always kind to everyone.",
        "I have a lot of work to finish by tomorrow.",
        "The food at the restaurant was delicious.",
        "They are planning to visit Paris next summer.",
        "My dog loves playing in the park.",
        "The concert was absolutely amazing.",
        "I don’t understand the point of this game.",
        "She is learning how to cook Italian dishes.",
        "He was thrilled to receive the promotion.",
        "The book offers valuable insights into human psychology.",
        "I find this topic incredibly boring.",
        "The car is brand new and drives smoothly.",
        "They moved to a new house last month.",
        "The painting is a masterpiece of art.",
        "I am grateful for all the help you provided.",
        "The customer service team was very helpful.",
        "He struggled to finish the project on time.",
        "The cake was too sweet for my taste.",
        "This park is a great place for a picnic.",
        "She felt nervous before the big exam.",
        "The news about the earthquake shocked everyone.",
        "We should recycle more to protect the environment.",
        "The party last night was a lot of fun.",
        "He apologized for his mistakes.",
        "The company announced record-breaking profits.",
        "I look forward to seeing you again.",
        "Traveling to new countries is always exciting."
    ],
    "es": [
        "El clima está hermoso hoy.",
        "Me encanta comer pizza los fines de semana.",
        "Ella es muy talentosa tocando el piano.",
        "Esta es la mejor película que he visto.",
        "El producto superó mis expectativas.",
        "Estoy muy decepcionado con el servicio.",
        "El nuevo iPhone es demasiado caro.",
        "Él siempre es amable con todos.",
        "Tengo mucho trabajo que terminar para mañana.",
        "La comida en el restaurante estaba deliciosa.",
        "Están planeando visitar París el próximo verano.",
        "Mi perro ama jugar en el parque.",
        "El concierto fue absolutamente increíble.",
        "No entiendo el punto de este juego.",
        "Ella está aprendiendo a cocinar platos italianos.",
        "Estaba emocionado por recibir la promoción.",
        "El libro ofrece valiosos conocimientos sobre psicología humana.",
        "Encuentro este tema increíblemente aburrido.",
        "El coche es completamente nuevo y funciona suavemente.",
        "Se mudaron a una casa nueva el mes pasado.",
        "La pintura es una obra maestra del arte.",
        "Estoy agradecido por toda la ayuda que me diste.",
        "El equipo de atención al cliente fue muy útil.",
        "Luchó por terminar el proyecto a tiempo.",
        "El pastel era demasiado dulce para mi gusto.",
        "Este parque es un gran lugar para un picnic.",
        "Se sintió nerviosa antes del gran examen.",
        "La noticia sobre el terremoto conmocionó a todos.",
        "Deberíamos reciclar más para proteger el medio ambiente.",
        "La fiesta de anoche fue muy divertida.",
        "Él se disculpó por sus errores.",
        "La empresa anunció beneficios récord.",
        "Espero verte de nuevo."
    ],
    "zh": [
        "今天的天气很好。",
        "我周末喜欢吃披萨。",
        "她弹钢琴很有天赋。",
        "这是我看过的最好的电影。",
        "产品超出了我的期望。",
        "我对这个服务很失望。",
        "新的iPhone太贵了。",
        "他对每个人都很友善。",
        "我有很多工作要在明天完成。",
        "餐厅的食物很好吃。",
        "他们计划明年夏天去巴黎。",
        "我的狗喜欢在公园玩耍。",
        "音乐会非常精彩。",
        "我不明白这个游戏的意义。",
        "她正在学习如何做意大利菜。",
        "他收到晋升后很兴奋。",
        "这本书提供了人类心理学的宝贵见解。",
        "我觉得这个话题非常无聊。",
        "这辆车是全新的，开起来很顺畅。",
        "他们上个月搬进了一所新房子。",
        "这幅画是一件艺术杰作。",
        "我很感激你提供的所有帮助。",
        "客户服务团队非常乐于助人。",
        "他努力按时完成项目。",
        "蛋糕对我来说太甜了。",
        "这个公园是野餐的好地方。",
        "考试前她感到很紧张。",
        "地震的消息震惊了所有人。",
        "我们应该更多地回收以保护环境。",
        "昨晚的派对很有趣。",
        "他为自己的错误道歉了。",
        "公司宣布创纪录的利润。",
        "我期待再次见到你。"
    ]
}

In [22]:
ground_truth = {
    "en": [
        "positive", "positive", "positive", "positive", "positive", 
        "negative", "negative", "positive", "neutral", "positive",
        "positive", "positive", "positive", "neutral", "positive",
        "positive", "positive", "neutral", "positive", "neutral",
        "positive", "positive", "positive", "negative", "negative",
        "positive", "negative", "negative", "positive", "positive",
        "positive", "positive", "positive", "positive"
    ],
    "es": [
        "positive", "positive", "positive", "positive", "positive",
        "negative", "negative", "positive", "neutral", "positive",
        "positive", "positive", "positive", "neutral", "positive",
        "positive", "positive", "neutral", "positive", "neutral",
        "positive", "positive", "positive", "negative", "negative",
        "positive", "negative", "negative", "positive", "positive",
        "positive", "positive", "positive"
    ],
    "zh": [
        "positive", "positive", "positive", "positive", "positive",
        "negative", "negative", "positive", "neutral", "positive",
        "positive", "positive", "positive", "neutral", "positive",
        "positive", "positive", "neutral", "positive", "neutral",
        "positive", "positive", "positive", "negative", "negative",
        "positive", "negative", "negative", "positive", "positive",
        "positive", "positive", "positive"
    ]
}


In [23]:
ibm_results = {}
total_correct = 0
total_predictions = 0
for lang_code, lang_name in languages.items():
    ibm_results[lang_code] = []

    for i, sentence in enumerate(sentences[lang_code]):
        # IBM Watson
        ibm_sentiment = analyze_sentiment_ibm(sentence, lang_code)
        ibm_results[lang_code].append(ibm_sentiment)

        if ibm_sentiment == ground_truth[lang_code][i]:
            total_correct += 1
        total_predictions += 1

accuracy_ibm = (total_correct / total_predictions) * 100

In [27]:
google_results = {}
total_correct = 0
total_predictions = 0
for lang_code, lang_name in languages.items():
    google_results[lang_code] = []

    for i, sentence in enumerate(sentences[lang_code]):
        # IBM Watson
        google_sentiment = analyze_sentiment_google(sentence, lang_code)
        google_results[lang_code].append(google_sentiment)

        if google_sentiment == ground_truth[lang_code][i]:
            total_correct += 1
        total_predictions += 1

accuracy_google = (total_correct / total_predictions) * 100

In [28]:
aws_results = {}
total_correct = 0
total_predictions = 0
for lang_code, lang_name in languages.items():
    aws_results[lang_code] = []

    for i, sentence in enumerate(sentences[lang_code]):
        # IBM Watson
        aws_sentiment = analyze_sentiment_aws(sentence, lang_code)
        aws_results[lang_code].append(aws_sentiment)

        if aws_sentiment == ground_truth[lang_code][i]:
            total_correct += 1
        total_predictions += 1


accuracy_aws = (total_correct / total_predictions) * 100

In [29]:
for lang_code, lang_name in languages.items():
    print(f"\n{lang_name}:")
    print(f"\nIBM Watson NLU Accuracy: {accuracy_ibm:.2f}%")
    print(f"\nGoogle Cloud Accuracy: {accuracy_google:.2f}%")
    print(f"\nAWS Comprehend Accuracy: {accuracy_aws:.2f}%")


English:

IBM Watson NLU Accuracy: 70.00%

Google Cloud Accuracy: 80.00%

AWS Comprehend Accuracy: 74.00%

Spanish:

IBM Watson NLU Accuracy: 70.00%

Google Cloud Accuracy: 80.00%

AWS Comprehend Accuracy: 74.00%

Chinese:

IBM Watson NLU Accuracy: 70.00%

Google Cloud Accuracy: 80.00%

AWS Comprehend Accuracy: 74.00%
